# OBJECTNESS EVALUATION

The purpose of this Notebook is to evaluate the performance of different generic object proposal approaches on high-resultion satellite data

In [1]:
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader

import datasets
from helpers import io

## Settings

In [2]:
DATA_PATH = "/home/jovyan/work/DENMARK/512x512"

## Load Data

In [3]:
images_path = os.path.join(DATA_PATH, 'image_sets_trees', 'eval_shapes.txt')
images = [name.replace("\n","") for name in io.readText(images_path)]

In [11]:
dataset = datasets.getDataset(name = "denmark_shapes", 
                              path = DATA_PATH,
                              images = images,
                              object_type = 'trees_eval',
                              n_classes = 2,
                              transform = None)

sampler = torch.utils.data.RandomSampler(dataset)
loader = DataLoader(dataset, sampler = sampler, batch_size = 1, drop_last = True, num_workers = 1)

## Eval COB

In [9]:
from models.cobnet import COBNet
from helpers.cob.dataset import COBtransform

means = [0.492, 0.475, 0.430]
stds = [0.176, 0.173, 0.176]
transform = COBtransform(means, stds, 512)

cob_model = COBNet()
cob_model.load_state_dict(torch.load("/home/jovyan/work/runs/X_COBNET/cp_or.pth.tar"))

<All keys matched successfully>

In [12]:
cob_loss_running = 0
cob_mIoU_running = 0

for batch in tqdm(loader):
    image = batch["images"].numpy()
    shapes = batch["shapes"].squeeze().numpy()
    
    # transforms
    image_cob = transform(images = image)[0]
    image_cob = np.stack((image_cob[:,:,0], image_cob[:,:,1], image_cob[:,:,2]), axis = 0)
    cob_tensor = torch.tensor(image_cob[np.newaxis, ...]).float()
    
    # model run
    cob_model.eval()
    with torch.no_grad():
        out_cob = cob_model(cob_tensor)
    
    cob_preds = out_cob['y_fine'].sigmoid().squeeze()
    
    # eval with BCELoss
    criterion = torch.nn.BCELoss()
    loss1 = criterion(cob_preds, torch.from_numpy(shapes.astype(np.float32)))
    loss0 = criterion(1 - cob_preds, torch.from_numpy(shapes.astype(np.float32)))
    ### NOTE: shape area should be as homogeneous as possible, not as low/high as possible -> take better score
    if loss1 < loss0:
        cob_loss_running += loss1.item()
    else:
        cob_loss_running += loss0.item()
    
    # eval with mIoU
    shapes_mask = shapes == 1
    cob_preds_np = torch.round(cob_preds).numpy()
    cob_mask1 = cob_preds_np == 1
    cob_mask0 = cob_preds_np == 0
    intersection1 = np.logical_and(cob_mask1, shapes_mask).sum()
    intersection0 = np.logical_and(cob_mask0, shapes_mask).sum()
    
    if intersection1 > intersection0:
        union = np.logical_or(cob_mask1, shapes_mask).sum()
        cob_mIoU_running += intersection1 / union
    else:
        union = np.logical_or(cob_mask0, shapes_mask).sum()
        cob_mIoU_running += intersection0 / union
        

cob_loss = cob_loss_running / len(loader)
cob_mIoU = cob_mIoU_running / len(loader)
print(f'COB Scores: \n -BCELoss: {cob_loss}\n -mIoU: {cob_mIoU}')

  0%|          | 0/20 [00:00<?, ?it/s]

COB Scores: 
 -BCELoss: 0.46418856233358385
 -mIoU: 0.08872219261236919


## Eval WTP

In [13]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath('/home/jovyan/work/ma/helpers/objectness', nargout=0)
eng.addpath('/home/jovyan/work/ma/helpers/objectness/pff_segment', nargout=0)
eng.addpath('/home/jovyan/work/ma/helpers/objectness/MEX', nargout=0)

In [14]:
wtp_loss_running = 0
wtp_mIoU_running = 0

for batch in tqdm(loader):
    image = batch['meta']['path'][0]
    shapes = batch["shapes"].squeeze().numpy()
    
    # matlab function run
    wtp_preds = np.asarray(eng.getHeatMap(image, 100))
    wtp_preds = torch.from_numpy(wtp_preds.astype(np.float32))
    
    # eval with BCELoss
    criterion = torch.nn.BCELoss()
    loss1 = criterion(wtp_preds, torch.from_numpy(shapes.astype(np.float32)))
    loss0 = criterion(1 - wtp_preds, torch.from_numpy(shapes.astype(np.float32)))
    ### NOTE: shape area should be as homogeneous as possible, not as low/high as possible -> take better score
    if loss1 < loss0:
        wtp_loss_running += loss1.item()
    else:
        wtp_loss_running += loss0.item()
        
    # eval with mIoU
    shapes_mask = shapes == 1
    wtp_preds_np = torch.round(wtp_preds).numpy()
    wtp_mask1 = wtp_preds_np == 1
    wtp_mask0 = wtp_preds_np == 0
    intersection1 = np.logical_and(wtp_mask1, shapes_mask).sum()
    intersection0 = np.logical_and(wtp_mask0, shapes_mask).sum()
    
    if intersection1 > intersection0:
        union = np.logical_or(wtp_mask1, shapes_mask).sum()
        wtp_mIoU_running += intersection1 / union
    else:
        union = np.logical_or(wtp_mask0, shapes_mask).sum()
        wtp_mIoU_running += intersection0 / union
        
    
wtp_loss = wtp_loss_running / len(loader)
wtp_mIoU = wtp_mIoU_running / len(loader)
print(f'WTP Scores: \n -BCELoss: {wtp_loss}\n -mIoU: {wtp_mIoU}')

  0%|          | 0/20 [00:00<?, ?it/s]

WTP Scores: 
 -BCELoss: 0.5484897948801517
 -mIoU: 0.09084602931774151


In [15]:
eng.quit()

## Eval CAM

In [ ]:
## load helper shit

In [ ]:
cam_loss_running = 0
cam_mIoU_running = 0

for batch in tqdm(loader):
    image = batch['images'].numpy()
    shapes = batch["shapes"].squeeze().numpy()
    
    # run estimation method
    cam_preds = ??? # needs to return [height x width] matrix in 0 - 1 range (sigmoid) as tensor
    
    # eval with BCELoss
    criterion = torch.nn.BCELoss()
    loss1 = criterion(cam_preds, torch.from_numpy(shapes.astype(np.float32)))
    loss0 = criterion(1 - cam_preds, torch.from_numpy(shapes.astype(np.float32)))
    ### NOTE: shape area should be as homogeneous as possible, not as low/high as possible -> take better score
    if loss1 < loss0:
        cam_loss_running += loss1.item()
    else:
        cam_loss_running += loss0.item()
        
    # eval with mIoU
    shapes_mask = shapes == 1
    cam_preds_np = torch.round(cam_preds).numpy()
    cam_mask1 = cam_preds_np == 1
    cam_mask0 = cam_preds_np == 0
    intersection1 = np.logical_and(cam_mask1, shapes_mask).sum()
    intersection0 = np.logical_and(cam_mask0, shapes_mask).sum()
    
    if intersection1 > intersection0:
        union = np.logical_or(cam_mask1, shapes_mask).sum()
        cam_mIoU_running += intersection1 / union
    else:
        union = np.logical_or(cam_mask0, shapes_mask).sum()
        cam_mIoU_running += intersection0 / union
        
    
cam_loss = cam_loss_running / len(loader)
cam_mIoU = cam_mIoU_running / len(loader)
print(f'WTP Scores: \n -BCELoss: {cam_loss}\n -mIoU: {cam_mIoU}')

# EXPERIMENTATION ZONE

In [ ]:
dataiter = iter(loader)
batch = dataiter.next()

In [ ]:
image = batch["images"].numpy()
shapes = batch["shapes"].squeeze().numpy()

In [ ]:
input = torch.randn((3, 250, 250), requires_grad=True)
target = torch.empty((3, 250, 250)).random_(2)

In [ ]:
# eval with BCELoss
criterion = torch.nn.BCELoss()
loss = criterion(input.sigmoid(), target)

In [ ]:
loss1 = criterion(input.sigmoid()[0], target[0])
loss2 = criterion(input.sigmoid()[1], target[1])
loss3 = criterion(input.sigmoid()[2], target[2])

In [ ]:
from PIL import Image, ImageDraw
img = Image.new('L', (256, 256), 0)
inb1 = np.array(img)
inb2 = torch.from_numpy(inb1).long()
inb3 = inb2.unsqueeze(0) # adds 1 dimension in front [256 x 256] -> [1 x 256 x 256]
shape = inb3.cuda()